In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
    'start':'1',
    'limit':'15',
    'convert':'USD'
}
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'dbea9faa-0ca7-4395-8945-76b533eab56b',
}

session = Session()
session.headers.update(headers)

try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)

In [ ]:
type(data)

In [ ]:
import pandas as pd


# Allows us to see all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Normalize and formats the data in a dataframe

df = pd.json_normalize(data['data'])

# Add a timestamp column to the dataframe
df['timestamp'] = pd.to_datetime('now', utc=True)
df

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
        'start':'1',
        'limit':'15',
        'convert':'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': 'dbea9faa-0ca7-4395-8945-76b533eab56b',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        # print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        
# To use a dataframe
        
    df2 = pd.json_normalize(data['data'])
    df2['Timestamp'] = pd.to_datetime('now', utc=True)
    df = pd.concat([df, df2], ignore_index=True)
    
        
# To create a CSV file and append data to it   

    # df = pd.json_normalize(data['data'])
    # df['timestamp'] = pd.to_datetime('now', utc=True)
    # df
    
    # if not os.path.isfile(r'C:\Users\dkbal\OneDrive\Documents\Python Scripts\API.csv'):
    #     df.to_csv(r'C:\Users\dkbal\OneDrive\Documents\Python Scripts\API.csv', header='column_names')
    # else:
    #     df.to_csv(r'C:\Users\dkbal\OneDrive\Documents\Python Scripts\API.csv', mode='a', header=False)

In [ ]:
import os
from time import time
from time import sleep

for i in range(333):
    api_runner()
    print("API Runner Completed")
    sleep(60) # sleep for 1 minute
exit()

In [ ]:
# To read the data from a CSV if using CSV instead of df

df2 = pd.read_csv(r'C:\Users\dkbal\OneDrive\Documents\Python Scripts\API.csv')
df2

In [ ]:
df

In [ ]:
# Change data in scientific notation to numbers

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df

In [ ]:
# Look at coin trends over time

df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d']].mean()
df3

In [ ]:
df4 = df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
# Change series back to dataframe

df5 = df4.to_frame(name='values')
df5

In [ ]:
df5.count()

In [ ]:
# Re-index the dataframe 

df6 = df5.reset_index()
df6

In [ ]:
# Change column 'level_1' name to 'percent_change'

df7 = df6.rename(columns={'level_1': 'percent_change'})
df7

In [ ]:
# Change data in percent_change column to be more readible

df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])
df7

In [ ]:
import seaborn as sns
import matplotlib as plt

In [ ]:
sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')

In [ ]:
# create dataframe with the columns that are wanted

df8 = df[['name','quote.USD.price','timestamp']]
df8 = df8.query("name == 'Bitcoin'")
df8

In [ ]:
sns.set_theme(style='darkgrid')

sns.lineplot(x='timestamp',y='quote.USD.price', data=df8)